<a href="https://colab.research.google.com/github/samuelbernsen08/CS615/blob/main/CS615_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Text embedding way

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
# Load the pre-trained VGG16 model
vgg_conv = kb.applications.vgg16.VGG16(
    weights = "imagenet",
    # include_top = False, # only including the convolution and pooling layers, not the dense FF layers
    input_shape = (224,224,3)
)

In [ ]:
# Load the image for inference
img_paths = ['/content/997_911_gt2rs.jpg', '/content/snick.jpg','/content/DSC_6846.JPG','/content/DSC_9220.JPG','/content/playfulsherbie1.jpg','/content/Sprite_LeBron_Commercial.jpg']
images = []
x = []
predictions = [None] * len(img_paths)  # Initialize the predictions list with the same length as the x list
decoded_predictions = [None] * len(img_paths)  # Initialize the predictions list with the same length as the x list
storage = []
for i in range(len(img_paths)):
    images.append(image.load_img(img_paths[i], target_size=(224, 224)))  # VGG16 input size is (224, 224)

    # Preprocess the image
    x.append(image.img_to_array(images[i]))
    x[i] = np.expand_dims(x[i], axis=0)
    x[i] = preprocess_input(x[i])

    # Run inference
    predictions[i] = vgg_conv.predict(x[i])

    # Decode the predictions
    decoded_predictions[i] = decode_predictions(predictions[i], top=1)[0]  # Get the top predicted classes

# Grab the second entries in each list
labels = [sublist[0][1] for sublist in decoded_predictions]

for i in range(len(labels)):
    storage.append([img_paths[i], labels[i]])

1/1 [==============================] - 1s 585ms/step


In [ ]:
decoded_predictions

[[('n03100240', 'convertible', 0.6443795)],
 [('n02951585', 'can_opener', 0.06589259)],
 [('n03877472', 'pajama', 0.11703915)],
 [('n09332890', 'lakeside', 0.48314404)],
 [('n02123159', 'tiger_cat', 0.22071552)],
 [('n02927161', 'butcher_shop', 0.2739127)]]

In [ ]:
# Load GloVe word vectors
def load_glove_vectors(glove_file):
    word_vectors = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            word_vectors[word] = vector
    return word_vectors

def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

# Function to embed a string using GloVe
def embed_string(string, word_vectors):
    words = string.split()
    embedding = np.zeros(word_vectors['the'].shape)  # Initialize embedding with zeros
    for word in words:
        if word in word_vectors:
            embedding += word_vectors[word]
    return embedding / len(words)  # Normalize by the number of words


glove_file = '/content/glove.6B.50d.txt'  # Path to the GloVe file
word_vectors = load_glove_vectors(glove_file)

In [ ]:
def remove_underscores(string):
    return string.replace("_", " ")

In [ ]:
strings = []
for i in range(len(labels)):
  strings.append(remove_underscores(storage[i][1]))

In [ ]:
embedding_vectors = []
for i in range(len(strings)):
  embedding_vectors.append(embed_string(strings[i], word_vectors))

# similarities = []
# for i in range(len(embedding_vectors)-1):
#   for j in range(len(embedding_vectors)-1):
#     if i != (j+1):
#       similarities.append(cosine_similarity(embedding_vectors[i], embedding_vectors[j+1]))

similarities = []
for i in range(len(embedding_vectors)-1):
  similarities.append(cosine_similarity(embedding_vectors[0], embedding_vectors[i+1]))

print(similarities)

[0.06552728130699631, 0.1586576165758021, 0.12359108221283541, 0.15830228340621189, 0.09819478733086058]


In [ ]:
highest_similarity = max(similarities)
highest_similarity_index = similarities.index(highest_similarity)

In [ ]:
highest_similarity_index

1

In [ ]:
next_image = storage[highest_similarity_index+1][0]
next_image

'/content/DSC_6846.JPG'



---



In [ ]:
highest_similarity_index = -1
p = 0
for k in range(len(storage)):
    similarities = []
    for i in range(len(storage)):
      if (i+k+1) <= len(storage) - 1:
        similarities.append(cosine_similarity(embedding_vectors[k], embedding_vectors[i+k+1]))
    highest_similarity = max(similarities)
    highest_similarity_index = similarities.index(highest_similarity) + 1

    if k < len(storage)-1:
      temp_k1 = embedding_vectors[1]
      temp_stor_k1 = storage[1]
      temp_high = embedding_vectors[highest_similarity_index+1]
      temp_stor_high = storage[highest_similarity_index+1]

      embedding_vectors[1] = embedding_vectors[highest_similarity_index+1]
      embedding_vectors[highest_similarity_index] = temp_k1
      temp_high = temp_k1
      temp_stor_high = temp_stor_k1

      storage[1] = storage[highest_similarity_index+1]
      storage[highest_similarity_index+1] = temp_stor_k1
      temp_stor_high = temp_stor_k1

      p += 1
print(p)

ValueError: max() arg is an empty sequence

In [ ]:
for k in range(len(storage)):
    similarities = []
    for i in range(len(storage)):
        if i != k:
            similarity = cosine_similarity(embedding_vectors[k], embedding_vectors[i])
            similarities.append(similarity)
    highest_similarity_index = similarities.index(max(similarities)) + 1

    # Swap embedding vectors and corresponding storage entries
    temp_k = embedding_vectors[k].copy()
    embedding_vectors[k] = embedding_vectors[highest_similarity_index].copy()
    embedding_vectors[highest_similarity_index] = temp_k

    temp_stor = storage[k]
    storage[k] = storage[highest_similarity_index]
    storage[highest_similarity_index] = temp_stor

print(p)


6


In [ ]:
storage

[['/content/DSC_6846.JPG', 'pajama'],
 ['/content/997_911_gt2rs.jpg', 'convertible'],
 ['/content/playfulsherbie1.jpg', 'tiger_cat'],
 ['/content/snick.jpg', 'can_opener'],
 ['/content/Sprite_LeBron_Commercial.jpg', 'butcher_shop'],
 ['/content/DSC_9220.JPG', 'lakeside']]

In [ ]:
for k in range(len(storage)):
    similarities = []
    for i in range(len(storage)-1):
      if (i+k+1) <= len(storage) - 1:
        similarities.append(cosine_similarity(embedding_vectors[k], embedding_vectors[i+k+1]))
    highest_similarity = max(similarities)
    highest_similarity_index = similarities.index(highest_similarity)

ValueError: max() arg is an empty sequence

In [ ]:
highest_similarity_index = -1

for k in range(len(storage)):
    similarities = []
    i = len(storage) - 1
    while i > 1:
      similarities.append(cosine_similarity(embedding_vectors[k], embedding_vectors[i]))
    highest_similarity = max(similarities)
    highest_similarity_index = similarities.index(highest_similarity)

    if k < len(storage)-1:
      temp_k1 = embedding_vectors[1]
      temp_stor_k1 = storage[1]
      temp_high = embedding_vectors[highest_similarity_index+1]
      temp_stor_high = storage[highest_similarity_index+1]

      embedding_vectors[1] = embedding_vectors[highest_similarity_index+1]
      embedding_vectors[highest_similarity_index] = temp_k1
      temp_high = temp_k1
      temp_stor_high = temp_stor_k1

      storage[1] = storage[highest_similarity_index+1]
      storage[highest_similarity_index+1] = temp_stor_k1
      temp_stor_high = temp_stor_k1

storage

In [ ]:
similarities = []
for i in range(len(storage)-1):
  highest_similarity_index = 0
  similarities.append(cosine_similarity(embedding_vectors[highest_similarity_index], embedding_vectors[i+1]))

highest_similarity = max(similarities)
highest_similarity_index = similarities.index(highest_similarity)

temp_k1 = embedding_vectors[1]
temp_stor_k1 = storage[1]
temp_high = embedding_vectors[highest_similarity_index+1]
temp_stor_high = storage[highest_similarity_index+1]

embedding_vectors[1] = embedding_vectors[highest_similarity_index+1]
storage[1] = storage[highest_similarity_index+1]
storage[highest_similarity_index+1] = temp_stor_k1
temp_stor_high = temp_stor_k1

highest_similarity_index

3

In [ ]:
storage

[['/content/playfulsherbie1.jpg', 'tiger_cat'],
 ['/content/snick.jpg', 'can_opener'],
 ['/content/997_911_gt2rs.jpg', 'convertible'],
 ['/content/DSC_9220.JPG', 'lakeside'],
 ['/content/DSC_6846.JPG', 'pajama']]

In [ ]:
embedding_vectors = []
for i in range(len(strings)):
  embedding_vectors.append(embed_string(strings[i], word_vectors))

# similarities = []
# for i in range(len(embedding_vectors)-1):
#   for j in range(len(embedding_vectors)-1):
#     if i != (j+1):
#       similarities.append(cosine_similarity(embedding_vectors[i], embedding_vectors[j+1]))

similarities = []
for i in range(len(embedding_vectors)-1):
  similarities.append(cosine_similarity(remove_underscores(storage[0][1]), remove_underscores(storage[i+1][1])))

print(similarities)

In [ ]:
cat = embed_string("tiger cat", word_vectors)
lakeside = embed_string("lakeside", word_vectors)
convertible = embed_string("convertible", word_vectors)
can_opener = embed_string("can opener", word_vectors)
pajama = embed_string("pajama", word_vectors)
butcher = embed_string("butcher shop", word_vectors)

print(cosine_similarity(cat,lakeside))
print(cosine_similarity(cat,convertible))
print(cosine_similarity(cat,can_opener))
print(cosine_similarity(cat,pajama))
print(cosine_similarity(cat,butcher))

0.08192158007767039
0.15830228340621189
0.5186216978940044
0.34653582721914733
0.39043469117244045


In [ ]:
print(cosine_similarity(lakeside,cat))
print(cosine_similarity(lakeside,convertible))
print(cosine_similarity(lakeside,can_opener))
print(cosine_similarity(lakeside,pajama))
print(cosine_similarity(lakeside,butcher))

0.08192158007767039
0.12359108221283541
-0.004699733250499103
0.08061996728599555
0.24157731609041283


In [ ]:
print(cosine_similarity(pajama,lakeside))
print(cosine_similarity(pajama,cat))
print(cosine_similarity(pajama,can_opener))
print(cosine_similarity(pajama,convertible))
print(cosine_similarity(pajama,butcher))

0.08061996728599555
0.34653582721914733
0.007765167446413493
0.1586576165758021
0.161078399447504


In [ ]:
print(cosine_similarity(can_opener,convertible))
print(cosine_similarity(can_opener,cat))
print(cosine_similarity(can_opener,pajama))
print(cosine_similarity(can_opener,lakeside))
print(cosine_similarity(can_opener,butcher))

0.06552728130699631
0.5186216978940044
0.007765167446413493
-0.004699733250499103
0.4202344306177217


In [ ]:
print(cosine_similarity(convertible,can_opener))
print(cosine_similarity(convertible,cat))
print(cosine_similarity(convertible,pajama))
print(cosine_similarity(convertible,lakeside))
print(cosine_similarity(convertible,butcher))

0.06552728130699631
0.15830228340621189
0.1586576165758021
0.12359108221283541
0.09819478733086058


In [ ]:
similarities

[0.08192158007767039,
 0.08192158007767039,
 0.08192158007767039,
 0.15830228340621189]



---



Image feature way

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image

# Load VGG16 model
vgg_model = VGG16(weights='imagenet', include_top=False)

# Function to preprocess image and extract features using VGG16
def extract_features(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    features = vgg_model.predict(img)
    features = features.flatten()  # Flatten to get a 1D feature vector
    return features

# List of image paths
image_paths = ['/content/997_911_gt2rs.jpg', '/content/snick.jpg', '/content/DSC_6846.JPG', '/content/DSC_9220.JPG', '/content/playfulsherbie1.jpg']

# Extract features for each image
features_list = [extract_features(img_path) for img_path in image_paths]

# Calculate cosine similarity between every pair of images
cosine_similarities = cosine_similarity(features_list, features_list)

# Print cosine similarity matrix
print(cosine_similarities)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the pre-trained VGG16 model
vgg_conv = tf.keras.applications.vgg16.VGG16(
    weights="imagenet",
    # include_top=False, # only including the convolution and pooling layers, not the dense FF layers
    input_shape=(224, 224, 3)
)

# Load GloVe word vectors
def load_glove_vectors(glove_file):
    word_vectors = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            word_vectors[word] = vector
    return word_vectors

# Function to embed a string using GloVe
def embed_string(string, word_vectors):
    words = string.split()
    embedding = np.zeros(word_vectors['the'].shape)  # Initialize embedding with zeros
    for word in words:
        if word in word_vectors:
            embedding += word_vectors[word]
    return embedding / len(words)  # Normalize by the number of words

def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

glove_file = '/content/glove.6B.50d.txt'  # Path to the GloVe file
word_vectors = load_glove_vectors(glove_file)

# Load the image for inference
img_paths = ['/content/997_911_gt2rs.jpg', '/content/snick.jpg','/content/DSC_6846.JPG','/content/DSC_9220.JPG','/content/playfulsherbie1.jpg']
images = []
x = []
predictions = [None] * len(img_paths)  # Initialize the predictions list with the same length as the x list
decoded_predictions = [None] * len(img_paths)  # Initialize the predictions list with the same length as the x list
storage = []
for i in range(len(img_paths)):
    images.append(image.load_img(img_paths[i], target_size=(224, 224)))  # VGG16 input size is (224, 224)

    # Preprocess the image
    x.append(image.img_to_array(images[i]))
    x[i] = np.expand_dims(x[i], axis=0)
    x[i] = preprocess_input(x[i])

    # Run inference
    predictions[i] = vgg_conv.predict(x[i])

    # Decode the predictions
    decoded_predictions[i] = decode_predictions(predictions[i], top=1)[0]  # Get the top predicted classes

# Grab the second entries in each list
labels = [sublist[0][1] for sublist in decoded_predictions]

for i in range(len(labels)):
    storage.append([img_paths[i], labels[i]])

# Prepare image labels
labels = [remove_underscores(sublist[1]) for sublist in storage]

# Embed image labels using GloVe
embedding_vectors = [embed_string(label, word_vectors) for label in labels]

# Calculate cosine similarity between embeddings
similarities = []
for i in range(len(embedding_vectors)-1):
    similarities.append(cosine_similarity(embedding_vectors[0], embedding_vectors[i+1]))

# Rearrange images based on similarities
sorted_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)
sorted_indices = [i+1 for i in sorted_indices]  # Adjust indices since we skipped the first image
sorted_images = [storage[i] for i in sorted_indices]

sorted_images  # This will give you the images organized by their similarities


1/1 [==============================] - 1s 561ms/step


[['/content/DSC_6846.JPG', 'pajama'],
 ['/content/playfulsherbie1.jpg', 'tiger_cat'],
 ['/content/DSC_9220.JPG', 'lakeside'],
 ['/content/snick.jpg', 'can_opener']]



---



##This is from help with Chat: It works by itself, as long as you import GLOVE and images with the paths copied correctly

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the pre-trained VGG16 model
vgg_conv = tf.keras.applications.vgg16.VGG16(
    weights="imagenet",
    # include_top=False, # only including the convolution and pooling layers, not the dense FF layers
    input_shape=(224, 224, 3)
)

# Load GloVe word vectors
def load_glove_vectors(glove_file):
    word_vectors = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            word_vectors[word] = vector
    return word_vectors

# Function to embed a string using GloVe
def embed_string(string, word_vectors):
    words = string.split()
    embedding = np.zeros(word_vectors['the'].shape)  # Initialize embedding with zeros
    for word in words:
        if word in word_vectors:
            embedding += word_vectors[word]
    return embedding / len(words)  # Normalize by the number of words

glove_file = '/content/glove.6B.50d.txt'  # Path to the GloVe file
word_vectors = load_glove_vectors(glove_file)

# Load the image for inference
img_paths = ['/content/997_911_gt2rs.jpg', '/content/snick.jpg','/content/DSC_6846.JPG','/content/DSC_9220.JPG','/content/playfulsherbie1.jpg']
images = []
x = []
predictions = [None] * len(img_paths)  # Initialize the predictions list with the same length as the x list
decoded_predictions = [None] * len(img_paths)  # Initialize the predictions list with the same length as the x list
storage = []
for i in range(len(img_paths)):
    images.append(image.load_img(img_paths[i], target_size=(224, 224)))  # VGG16 input size is (224, 224)

    # Preprocess the image
    x.append(image.img_to_array(images[i]))
    x[i] = np.expand_dims(x[i], axis=0)
    x[i] = preprocess_input(x[i])

    # Run inference
    predictions[i] = vgg_conv.predict(x[i])

    # Decode the predictions
    decoded_predictions[i] = decode_predictions(predictions[i], top=1)[0]  # Get the top predicted classes

# Grab the second entries in each list
labels = [sublist[0][1] for sublist in decoded_predictions]

for i in range(len(labels)):
    storage.append([img_paths[i], labels[i]])

# Prepare image labels
labels = [remove_underscores(sublist[1]) for sublist in storage]

# Embed image labels using GloVe
embedding_vectors = [embed_string(label, word_vectors) for label in labels]

# Calculate cosine similarity between embeddings
similarities = cosine_similarity(embedding_vectors, embedding_vectors)

# Rearrange images based on similarities
sorted_indices = similarities.argsort(axis=None)[::-1]

# Ensure sorted indices are within valid range
valid_sorted_indices = [i for i in sorted_indices if i < len(storage)]

# Obtain sorted images based on valid sorted indices
sorted_images = [storage[i] for i in valid_sorted_indices]

sorted_images  # This will give you the images organized by their similarities


1/1 [==============================] - 1s 583ms/step


[['/content/997_911_gt2rs.jpg', 'convertible'],
 ['/content/DSC_6846.JPG', 'pajama'],
 ['/content/playfulsherbie1.jpg', 'tiger_cat'],
 ['/content/DSC_9220.JPG', 'lakeside'],
 ['/content/snick.jpg', 'can_opener']]